<a href="https://colab.research.google.com/github/frankwillard/NBA-Hall-Of-Fame-Model/blob/main/Hall_of_Fame_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Packages and Data

In [54]:
# Load packages

from math import exp
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [55]:
# read in data
model_df = pd.read_csv("https://raw.githubusercontent.com/frankwillard/NBA-Hall-Of-Fame-Model/main/Scraped%20Player%20Data.csv", index_col=0)
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4977 entries, 0 to 4976
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          4977 non-null   object 
 1   Eligible        4977 non-null   int64  
 2   Position        4977 non-null   object 
 3   Hall_of_Fame    4977 non-null   int64  
 4   MVP             4977 non-null   int64  
 5   Finals_MVP      4977 non-null   int64  
 6   NBA_Champ       4977 non-null   int64  
 7   All_NBA         4977 non-null   int64  
 8   All_Defensive   4977 non-null   int64  
 9   Def_POY         4977 non-null   int64  
 10  All_Star        4977 non-null   int64  
 11  Scoring_Champ   4977 non-null   int64  
 12  TRB_Champ       4977 non-null   int64  
 13  AST_Champ       4977 non-null   int64  
 14  STL_Champ       4977 non-null   int64  
 15  BLK_Champ       4977 non-null   int64  
 16  All_ABA         4977 non-null   int64  
 17  ABA_Champ       4977 non-null   i

### Data Cleaning

In [56]:
model_df = model_df.replace(-999, np.nan)

In [57]:
# Reduce number of positions
model_df.loc[model_df['Position'] == 'Center/Forward', 'Position'] = 'Center'
model_df.loc[model_df['Position'].isin(['PointGuard', 'ShootingGuard', 'Guard/Forward']), 'Position'] = 'Guard'
model_df.loc[model_df['Position'].isin(['SmallForward', 'PowerForward', 'Forward/Guard', 'Forward/Center']), 'Position'] = 'Forward'

In [58]:
# Add ABA and NBA accolades
model_df['All_League'] = model_df['All_NBA'] + model_df['All_ABA']
model_df['Champ'] = model_df['NBA_Champ'] + model_df['ABA_Champ']

In [59]:
#Columns with NAs:
for col in model_df.columns:
  if len(model_df[model_df[col].isna()]) > 0:
    print(col, "-", len(model_df[model_df[col].isna()]))

3P_per_game - 1118
3PA_per_game - 1118
2P_per_game - 1118
2PA_per_game - 1118
ORB_per_game - 949
DRB_per_game - 949
TRB_per_game - 288
STL_per_game - 1180
BLK_per_game - 1180
GS_totals - 1689
FG%_totals - 34
3P_totals - 1118
3PA_totals - 1118
3P%_totals - 1627
2P_totals - 1118
2PA_totals - 1118
2P%_totals - 1162
eFG%_totals - 1146
FT%_totals - 241
ORB_totals - 949
DRB_totals - 949
TRB_totals - 288
STL_totals - 1180
BLK_totals - 1180
Trp_Dbl_totals - 4526
PER_advanced - 344
TS%_advanced - 29
OWS_advanced - 1
DWS_advanced - 1
WS_advanced - 1
WS/48_advanced - 344
OBPM_advanced - 1185
DBPM_advanced - 1185
BPM_advanced - 1185
VORP_advanced - 1183


In [60]:
# Columns with -999s
for col in model_df.columns:
  if len(model_df[model_df[col] == -999]) > 0:
    print(col, "-", len(model_df[model_df[col] == -999]))

In [61]:
# WHAT TO DO WITH NAs / -999

# Columns to drop:
# GS_totals, Trp_Dbl_totals, ORB_per_game, DRB_per_game, ORB_totals, DRB_totals, 3P%_totals, 2P%_totals, eFG%_totals, OWS_advanced, DWS_advanced, WS/48_advanced, OBPM_advanced, DBPM_advanced

# Columns to drop for now: (just for bare bones model, consider bringing back FG% FT% 2PT stuff at least)
# 3P_per_game, 3PA_per_game, 3P_totals, 3PA_totals
# FG%_totals (these players never took a shot)
# FT%_totals (these players never took a FT)
# Columns to take from FGM, FGA, etc.
# 2P_per_game, 2PA_per_game, 2P_totals, 2PA_totals

# Columns to fill with league average
# PER_advanced, VORP_advanced (consider some more advanced PER/VORP)

# Columns to make 0
# WS_advanced, BPM_advanced (consider some more advance imputation for BPM)

# Columns to make 0 or fill with mean (undecided):***
# TS%_advanced (these players never took a shot or free throw?)

# Columns to fill with mean (potentially by position):
# PTS_per_game, TRB_per_game, AST_per_game, STL_per_game, BLK_per_game, TRB_totals, AST_totals, STL_totals, BLK_totals

In [62]:
def fillNulls(model_df):
  cols_to_zero = ['WS_advanced', 'BPM_advanced', '3P_per_game', '3PA_per_game', '3P_totals', '3PA_totals', 'FG%_totals', 'FT%_totals', 'TS%_advanced']
  model_df[cols_to_zero] = model_df[cols_to_zero].fillna(0) # fill cols with 0
  
  cols_to_avg = ['PER_advanced', 'VORP_advanced', '3P%_totals', '2P%_totals', 'eFG%_totals']
  model_df[cols_to_avg] = model_df[cols_to_avg].fillna(model_df[cols_to_avg].mean()) # fill cols with avg
  
  cols_to_position_avg = ['TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game', 'TRB_totals', 'AST_totals', 'STL_totals', 'BLK_totals']
  model_df[cols_to_position_avg] = model_df.groupby("Position")[cols_to_position_avg].transform(lambda x: x.fillna(x.mean())) # fills cols with avg by position
  
  cols_to_fill = ['2P_per_game', '2PA_per_game', '2P_totals', '2PA_totals']
  cols_to_fill_with = ['FG_per_game', 'FGA_per_game', 'FG_totals', 'FGA_totals']
  model_df[cols_to_fill] = model_df[cols_to_fill].fillna(model_df[cols_to_fill_with]) # fill 2P shooting columns with FG columns

  return model_df
fillNulls(model_df)

,Player,Eligible,Position,Hall_of_Fame,MVP,Finals_MVP,NBA_Champ,All_NBA,All_Defensive,Def_POY,...,OWS_advanced,DWS_advanced,WS_advanced,WS/48_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced,All_League,Champ
0,Alaa Abdelnaby,1,Forward,0,0,0,0,0,0,0,...,0.7,4.1,4.8,0.072,-2.9,-0.9,-3.8,-1.500000,0,0
1,Zaid Abdul-Aziz,1,Center,0,0,0,0,0,0,0,...,5.9,11.6,17.5,0.076,0.6,-0.2,0.4,2.700000,0,0
2,Kareem Abdul-Jabbar,1,Center,1,6,2,6,15,11,0,...,178.9,94.5,273.4,0.228,4.1,1.6,5.7,85.700000,15,6
3,Mahmoud Abdul-Rauf,1,Guard,0,0,0,0,0,0,0,...,16.7,8.4,25.2,0.077,0.7,-1.5,-0.8,4.500000,0,0
4,Tariq Abdul-Wahad,1,Guard,0,0,0,0,0,0,0,...,-0.6,4.1,3.5,0.035,-2.6,-0.4,-3.0,-1.200000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,Jim Zoet,1,Center,0,0,0,0,0,0,0,...,-0.1,0.0,-0.1,-0.123,-5.6,0.2,-5.4,-0.100000,0,0
4973,Bill Zopf,1,Guard,0,0,0,0,0,0,0,...,-0.5,0.4,-0.1,-0.011,NaN,NaN,0.0,3.434634,0,0
4974,Ivica Zubac,0,Center,0,0,0,0,0,0,0,...,16.4,9.6,26.1,0.183,0.3,0.4,0.6,4.500000,0,0
4975,Matt Zunic,1,Guard,0,0,0,0,0,0,0,...,0.2,1.8,2.0,NaN,NaN,NaN,0.0,3.434634,0,0


### Feature Selection

In [63]:
model_df.columns

Index(['Player', 'Eligible', 'Position', 'Hall_of_Fame', 'MVP', 'Finals_MVP',
       'NBA_Champ', 'All_NBA', 'All_Defensive', 'Def_POY', 'All_Star',
       'Scoring_Champ', 'TRB_Champ', 'AST_Champ', 'STL_Champ', 'BLK_Champ',
       'All_ABA', 'ABA_Champ', 'ROY', 'FG_per_game', 'FGA_per_game',
       '3P_per_game', '3PA_per_game', '2P_per_game', '2PA_per_game',
       'FT_per_game', 'FTA_per_game', 'ORB_per_game', 'DRB_per_game',
       'TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game',
       'PTS_per_game', 'G_totals', 'GS_totals', 'FG_totals', 'FGA_totals',
       'FG%_totals', '3P_totals', '3PA_totals', '3P%_totals', '2P_totals',
       '2PA_totals', '2P%_totals', 'eFG%_totals', 'FT_totals', 'FTA_totals',
       'FT%_totals', 'ORB_totals', 'DRB_totals', 'TRB_totals', 'AST_totals',
       'STL_totals', 'BLK_totals', 'PTS_totals', 'Trp_Dbl_totals',
       'PER_advanced', 'TS%_advanced', 'OWS_advanced', 'DWS_advanced',
       'WS_advanced', 'WS/48_advanced', 'OBPM_advanced

In [72]:
# 'Finals_MVP', 'BLK_totals', 'VORP_advanced', 'STL_totals', 'TS%_advanced', 'All_Defensive', 'AST_Champ', 'TRB_totals', 'AST_totals'

# 'BPM_advanced', 'WS_advanced',
model_cols = ['Player', 'Eligible', 'Hall_of_Fame', 'Def_POY', 'All_Star', 'All_League', 'Champ', 'Scoring_Champ', # accolades
       'PTS_totals'] # stats
df = model_df[model_cols]
df

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals
0,Alaa Abdelnaby,1,0,0,0,0,0,0,1465
1,Zaid Abdul-Aziz,1,0,0,0,0,0,0,4557
2,Kareem Abdul-Jabbar,1,1,0,19,15,6,2,38387
3,Mahmoud Abdul-Rauf,1,0,0,0,0,0,0,8553
4,Tariq Abdul-Wahad,1,0,0,0,0,0,0,1830
...,...,...,...,...,...,...,...,...,...
4972,Jim Zoet,1,0,0,0,0,0,0,2
4973,Bill Zopf,1,0,0,0,0,0,0,118
4974,Ivica Zubac,0,0,0,0,0,0,0,3001
4975,Matt Zunic,1,0,0,0,0,0,0,273


In [109]:
eligible_df = df[df['Eligible'] == 1]
noneligible_df = df[df['Eligible'] == 0]

In [110]:
extraneous_players = ['Maurice Stokes', 'Bill Bradley', 'Toni Kukoč',
       'Calvin Murphy', 'Vlade Divac', 'Buddy Jeannette',
       'Dražen Petrović', 'Al Cervi', 'Arvydas Sabonis',
       'Šarūnas Marčiulionis', 'Dino Radja', 'Chuck Cooper',
       'Bob Houbregs']

eligible_df = eligible_df[~eligible_df['Player'].isin(extraneous_players)]

### Train/Test Split

In [111]:
y_eligible = eligible_df.iloc[:, 2].values

In [112]:
# Train test split
from sklearn.model_selection import train_test_split
X_training, X_validation, y_train, y_val = train_test_split(eligible_df, y_eligible, test_size = 0.25, random_state = 0)

In [118]:
X_train = X_training.iloc[:,3:].values
X_val = X_validation.iloc[:,3:].values

In [119]:
#X_eligible = eligible_df.iloc[:, 3:].values
X_test = noneligible_df.iloc[:, 3:]
y_test = noneligible_df.iloc[:, 2]

### Feature Scaling

In [120]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# Scale whole matrix of features to prevent information leakage
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

### Modeling

In [121]:
#from sklearn.preprocessing import PolynomialFeatures
#poly_reg = PolynomialFeatures(degree = 2)
#X_poly = poly_reg.fit_transform(X_train)

In [122]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 1)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Hyperparameter Optimization

In [ ]:
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

#from sklearn.model_selection import RepeatedStratifiedKFold
#from sklearn.model_selection import GridSearchCV

#solvers = ['newton-cg', 'lbfgs', 'liblinear']
#penalty = ['l2']
#c_values = [100, 10, 1.0, 0.1, 0.01]

In [ ]:
# define grid search
#grid = dict(solver=solvers,penalty=penalty,C=c_values)
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
#grid_result = grid_search.fit(X, y)

In [ ]:
# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

### Predictions

In [136]:
y_train_pred_probs = classifier.predict_proba(X_train)[:, 1]
y_train_pred = classifier.predict(X_train)

In [137]:
y_val_pred_probs = classifier.predict_proba(X_val)[:, 1]
y_val_pred = classifier.predict(X_val)

In [138]:
#eligible_df['pred'] = y_train_pred_probs
X_training['pred'] = y_train_pred_probs
X_validation['pred'] = y_val_pred_probs

### Borderline Correct Positive HOF Predictions

In [144]:
#eligible_df[(eligible_df['pred'] > 0.5) & (eligible_df['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)
X_training[(X_training['pred'] > 0.5) & (X_training['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
2673,Clyde Lovellette,1,1,0,4,1,3,0,11947,0.537291
2910,Dick McGuire,1,1,0,7,1,0,0,5921,0.564610
4698,Paul Westphal,1,1,0,5,4,1,0,12809,0.587713
2029,Lou Hudson,1,1,0,6,1,0,0,17940,0.601681
4481,Jack Twyman,1,1,0,6,2,0,0,15840,0.607734


In [145]:
X_validation[(X_validation['pred'] > 0.5) & (X_validation['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
3944,Charlie Scott,1,1,0,5,2,1,0,14837,0.513173
2211,Gus Johnson,1,1,0,5,4,1,0,10243,0.514441
289,Zelmo Beaty,1,1,0,5,2,1,0,15207,0.523854
4663,Chris Webber,1,1,0,5,5,0,0,17182,0.539958
1583,Gail Goodrich,1,1,0,5,1,1,0,19181,0.571233


### Under predictions

#### Expected Under Predictions:

<br/>

#### **Old players:**

<br/>


Bob Houbregs- A few seasons with mediocre stats, great college player but no NBA accolades and no other league. No clue why he is in. From 50s

<br/>

Chuck Cooper- 6.7 PPG and no accolades. Instrumental to NBA as broke the color barrier

<br/>

Al Cervi- 4 years of recorded stats with Syracuse (49-53). Dominated NBL with scoring champion, championship, 4 all league

<br/>

Buddy Jeannette	- From 40s, 3 seasons in BAA, but played in late 30s, early 40s with no stats for NBL- seeral first team/titles

<br/>
<br/>


#### **European players:**

<br/>

Dino Radja - Croatian player, only 4 years in NBA (with very good stats) but made it for Europe play

<br/>

Šarūnas Marčiulionis - 8 solid years in NBA, 8 years before in USSR league, made it for foreign/olympic play

<br/>

Arvydas Sabonis	- Many great years in Lithuania/Spain unaccounted for, solid 7 years in NBA in 30s


<br/>

Dražen Petrović	- 5 solid years in NBA, played in Yugoslavia/Spain


<br/>

Vlade Divac	- Many years in NBA with solid stats, no accolades. Did well in Yugoslav league, won Europa POY in 7 years before NBA

<br/>

Toni Kukoc- Very solid NBA career and picked up some championships. Played in Italy/Yugoslavia before (9 years). Won Europa POY, Euroscar POY, many Euroleage Championships

<br/>
<br/>


#### **Other:**

Bill Bradley- Not great individual stats, exceptional college player and 2 time champ but not above average. 70s so not that old

<br/>

Calvin Murphy- Very good pro career with not many accolades. Three time all american in college

<br/>

Maurice Stokes - From 50s, 3 years in NBA putting up 17-17 type numbers. Injured and paralyzed after three years with 3 all NBA. Good college player too

<br/>



#### **Legitimately wrong:**

Guy Rodgers- Long NBA career, 4x all star, 2x ast champ

Ralph Sampson- 6 solid years in NBA (of his 9), 4x all star, legend in college

Bill Walton - MVP, Legend in college, 2x all star, 2x all league. Can be fixed by adding MVP variable but may hurt with other players

Walt Bellamy- 4x all star, tons of points (43) and rebounds (12) over career

Players to look into:

 'Walt Bellamy',
       'Maurice Cheeks', 'Earl Monroe', 'Arnie Risen', 'Tom Gola',
       'Frank Ramsey', 'Wes Unseld', 'Andy Phillip', 'Bob Dandridge',
       'David Thompson'

In [127]:
#eligible_df[(eligible_df['pred'] < 0.5) & (eligible_df['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)
X_training[(X_training['pred'] < 0.5) & (X_training['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
3136,Chris Mullin,1,1,0,5,4,0,0,17911,0.494077
1781,Tim Hardaway,1,1,0,5,5,0,0,15373,0.487747
4754,Jamaal Wilkes,1,1,0,3,0,4,0,14644,0.481022
489,Carl Braun,1,1,0,5,2,1,0,10625,0.393080
4492,Wes Unseld,1,1,0,5,1,1,0,10624,0.331221
1561,Manu Ginóbili,1,1,0,2,2,4,0,14043,0.300182
3497,Andy Phillip,1,1,0,5,2,1,0,6384,0.283976
3068,Earl Monroe,1,1,0,4,1,1,0,17454,0.240461
308,Walt Bellamy,1,1,0,4,0,0,0,20941,0.120968
761,Maurice Cheeks,1,1,0,4,0,1,0,12195,0.116450


In [141]:
X_validation[(X_validation['pred'] < 0.5) & (X_validation['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
1848,Connie Hawkins,1,1,0,5,3,1,0,11528,0.484558
967,Bob Dandridge,1,1,0,4,1,2,0,15530,0.400248
4606,Bobby Wanzer,1,1,0,5,3,1,0,6924,0.355670
4376,David Thompson,1,1,0,5,3,0,0,13422,0.307682
1576,Tom Gola,1,1,0,5,1,1,0,7871,0.264826
2284,K.C. Jones,1,1,0,0,0,8,0,5011,0.263273
3730,Arnie Risen,1,1,0,4,1,2,0,7633,0.211208
989,Bob Davies,1,1,0,4,5,1,0,6594,0.208605
3627,Frank Ramsey,1,1,0,0,0,7,0,8378,0.166884
3878,Ralph Sampson,1,1,0,4,1,0,0,7039,0.034798


### Over predictions

Larry Foust deserves to be in HOF. Also has 94% rating on Bball ref

Amare Stoudemire (72% bball ref), Chauncey (84% on bball ref) are newer to ballot

Larry Costello is in HOF as Contributor- this led to confusion as one of his key contributions was his play. 71% on bball ref

Tom Sanders in HOF as contributor but 15% rating on bball ref

In [128]:
#eligible_df[(eligible_df['pred'] > 0.5) & (eligible_df['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)
X_training[(X_training['pred'] > 0.5) & (X_training['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
4240,Amar'e Stoudemire,1,0,0,6,5,0,0,15994,0.779046
1028,Walter Davis,1,0,0,6,2,0,0,19521,0.703386
2363,Shawn Kemp,1,0,0,6,3,0,0,15347,0.656775
351,Chauncey Billups,1,0,0,5,3,1,0,15802,0.606463
3298,Jermaine O'Neal,1,0,0,6,3,0,0,13309,0.601873
888,Larry Costello,1,0,0,6,1,1,0,8622,0.575209
1440,Donnie Freeman,1,0,0,5,4,1,0,12233,0.571484
2281,Jimmy Jones,1,0,0,6,3,0,0,11366,0.547006


In [142]:
X_validation[(X_validation['pred'] > 0.5) & (X_validation['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,All_League,Champ,Scoring_Champ,PTS_totals,pred
1412,Larry Foust,1,0,0,8,2,0,0,11198,0.91495


In [129]:
#eligible_df.sort_values(by='PER_advanced', ascending=False)

### Model Coefficients

In [146]:
for col, coef in zip(model_cols[2:], classifier.coef_[0]):
  print(f"{col}: {exp(coef)}")

Hall_of_Fame: 1.2280004836307257
Def_POY: 7.644193797426366
All_Star: 1.3683143247837941
All_League: 2.0359911750418727
Champ: 3.635924746492836
Scoring_Champ: 1.7094165190622375


In [131]:
#for col, coef in zip(poly_reg.get_feature_names(), classifier.coef_[0]):
#  print(f"{col}: {exp(coef)}")

### Confusion Matrix

In [133]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_train, y_train_pred)
#[00 01]
#[10 11]
print(cm)
accuracy_score(y_train, y_train_pred)

[[2956    8]
 [  12   84]]


0.9934640522875817

In [147]:
cm = confusion_matrix(y_val, y_val_pred)
#[00 01]
#[10 11]
print(cm)
accuracy_score(y_val, y_val_pred)

[[988   1]
 [ 10  21]]


0.9892156862745098